# Features Extraction

In [294]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import pywt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

## 1. Time Domain Features, Frequency Domain Features, T-F Domain Features Functions

In [217]:
def time_features_estimation(signal, frame, step):
    """
    Compute time features from signal using sliding window method.
    :param signal: numpy array signal.
    :param frame: sliding window size.
    :param step: sliding window step size.
    :return: time_features_matrix: narray matrix with the time features stacked by columns.
    """
    #variance = []
    rms = []
    #iemg = []
    mav = []
    #log_detector = []
    #wl = []
    #aac = []
    #dasdv = []
    zc = []
    #wamp = []
    #myop = []

    th = np.mean(signal) + 3 * np.std(signal)

    for i in range(frame, signal.size, step):
        x = signal[i - frame:i]

        #variance.append(np.var(x))
        rms.append(np.sqrt(np.mean(x ** 2)))
        #iemg.append(np.sum(abs(x)))  # Integral
        mav.append(np.sum(np.absolute(x)) / frame)  # Mean Absolute Value
        #log_detector.append(np.exp(np.sum(np.log10(np.absolute(x))) / frame))
        #wl.append(np.sum(abs(np.diff(x))))  # Wavelength
        #aac.append(np.sum(abs(np.diff(x))) / frame)  # Average Amplitude Change
       # dasdv.append(
        #    math.sqrt((1 / (frame - 1)) * np.sum((np.diff(x)) ** 2)))  # Difference absolute standard deviation value
        zc.append(zcruce(x, th))  # Zero-Crossing
       # wamp.append(wilson_amplitude(x, th))  # Willison amplitude
        #myop.append(myopulse(x, th))  # Myopulse percentage rate

    #time_features_matrix = np.column_stack((variance, rms, iemg, mav, log_detector, wl, aac, dasdv, zc, wamp, myop))
    #time_features_matrix = np.column_stack((rms, mav, zc))
    time_features_matrix = rms+mav+zc
    return time_features_matrix

In [3]:
def frequency_features_estimation(signal, fs, frame, step):
    """
    Compute frequency features from signal using sliding window method.
    :param signal: numpy array signal.
    :param fs: sampling frequency of the signal.
    :param frame: sliding window size
    :param step: sliding window step size
    :return: frequency_features_matrix: narray matrix with the frequency features stacked by columns.
    """

    fr = []
    mnp = []
    tot = []
    mnf = []
    mdf = []
    pkf = []

    for i in range(frame, signal.size, step):
        x = signal[i - frame:i]
        frequency, power = spectrum(x, fs)

        fr.append(frequency_ratio(frequency, power))  # Frequency ratio
        mnp.append(np.sum(power) / len(power))  # Mean power
        tot.append(np.sum(power))  # Total power
        mnf.append(mean_freq(frequency, power))  # Mean frequency
        mdf.append(median_freq(frequency, power))  # Median frequency
        pkf.append(frequency[power.argmax()])  # Peak frequency

    frequency_features_matrix = np.column_stack((fr, mnp, tot, mnf, mdf, pkf))

    return frequency_features_matrix

In [4]:
def wilson_amplitude(signal, th):
    x = abs(np.diff(signal))
    umbral = x >= th
    return np.sum(umbral)


def myopulse(signal, th):
    umbral = signal >= th
    return np.sum(umbral) / len(signal)


def spectrum(signal, fs):
    m = len(signal)
    n = next_power_of_2(m)
    y = np.fft.fft(signal, n)
    yh = y[0:int(n / 2 - 1)]
    fh = (fs / n) * np.arange(0, n / 2 - 1, 1)
    power = np.real(yh * np.conj(yh) / n)

    return fh, power


def frequency_ratio(frequency, power):
    power_low = power[(frequency >= 30) & (frequency <= 250)]
    power_high = power[(frequency > 250) & (frequency <= 500)]
    ULC = np.sum(power_low)
    UHC = np.sum(power_high)

    return ULC / UHC


def shannon(x):
    N = len(x)
    nb = 19
    hist, bin_edges = np.histogram(x, bins=nb)
    counts = hist / N
    nz = np.nonzero(counts)

    return np.sum(counts[nz] * np.log(counts[nz]) / np.log(2))


def zcruce(X, th):
    th = 0
    cruce = 0
    for cont in range(len(X) - 1):
        can = X[cont] * X[cont + 1]
        can2 = abs(X[cont] - X[cont + 1])
        if can < 0 and can2 > th:
            cruce = cruce + 1
    return cruce


def mean_freq(frequency, power):
    num = 0
    den = 0
    for i in range(int(len(power) / 2)):
        num += frequency[i] * power[i]
        den += power[i]

    return num / den


def median_freq(frequency, power):
    power_total = np.sum(power) / 2
    temp = 0
    tol = 0.01
    errel = 1
    i = 0

    while abs(errel) > tol:
        temp += power[i]
        errel = (power_total - temp) / power_total
        i += 1
        if errel < 0:
            errel = 0
            i -= 1

    return frequency[i]


def wavelet_energy(x, mother, nivel):
    coeffs = pywt.wavedecn(x, wavelet=mother, level=nivel)
    arr, _ = pywt.coeffs_to_array(coeffs)
    Et = np.sum(arr ** 2)
    cA = coeffs[0]
    Ea = 100 * np.sum(cA ** 2) / Et
    Ed = []

    for k in range(1, len(coeffs)):
        cD = list(coeffs[k].values())
        cD = np.asarray(cD)
        Ed.append(100 * np.sum(cD ** 2) / Et)

    return Ea, Ed


def next_power_of_2(x):
    return 1 if x == 0 else 2 ** (x - 1).bit_length()


def med_freq(f, P):
    Ptot = np.sum(P) / 2
    temp = 0
    tol = 0.01
    errel = 1
    i = 0

    while abs(errel) > tol:
        temp += P[i]
        errel = (Ptot - temp) / Ptot
        i += 1
        if errel < 0:
            errel = 0
            i -= 1

    return f[i]

In [5]:
def time_frequency_features_estimation(signal, frame, step):
    """
    Compute time-frequency features from signal using sliding window method.
    :param signal: numpy array signal.
    :param frame: sliding window size
    :param step: sliding window step size
    :return: h_wavelet: list
    """
    h_wavelet = []

    for i in range(frame, signal.size, step):
        x = signal[i - frame:i]

        E_a, E = wavelet_energy(x, 'db2', 4)
        E.insert(0, E_a)
        E = np.asarray(E) / 100

        h_wavelet.append(-np.sum(E * np.log2(E)))

    return h_wavelet

In [6]:
def features_estimation(signal, channel_name, fs, frame, step, plot=False):
    """
    Compute time, frequency and time-frequency features from signal.
    :param signal: numpy array signal.
    :param channel_name: string variable with the EMG channel name in analysis.
    :param fs: int variable with the sampling frequency used to acquire the signal
    :param frame: sliding window size
    :param step: sliding window step size
    :param plot: bolean variable to plot estimated features.
    :return: total_feature_matrix -- python Dataframe with .
    :return: features_names -- python list with
    """

    features_names = ['VAR', 'RMS', 'IEMG', 'MAV', 'LOG', 'WL', 'ACC', 'DASDV', 'ZC', 'WAMP', 'MYOP', "FR", "MNP", "TP",
                      "MNF", "MDF", "PKF", "WENT"]

    time_matrix = time_features_estimation(signal, frame, step)
    frequency_matrix = frequency_features_estimation(signal, fs, frame, step)
    time_frequency_matrix = time_frequency_features_estimation(signal, frame, step)
    total_feature_matrix = pd.DataFrame(np.column_stack((time_matrix, frequency_matrix, time_frequency_matrix)).T,
                                        index=features_names)

    print('EMG features were from channel {} extracted successfully'.format(channel_name))

    if plot:
        plot_features(signal, channel_name, fs, total_feature_matrix, step)

    return total_feature_matrix, features_names

In [7]:
def plot_features(signal, channel_name, fs, feature_matrix, step):
    """
    xxxs
    Argument:
    signal -- python numpy array representing recording of a signal.
    channel_name -- string variable with the EMG channel name in analysis.
    fs -- int variable with the sampling frequency used to acquire the signal.
    feature_matrix -- python Dataframe ...
    step -- int variable with the step size used in the sliding window method.
    """

    ts = np.arange(0, len(signal) / fs, 1 / fs)
    # for idx, f in enumerate(tfeatures.T):
    for key in feature_matrix.T:
        tf = step * (np.arange(0, len(feature_matrix.T[key]) / fs, 1 / fs))
        fig = plt.figure()

        ax = fig.add_subplot(111, label="1")
        ax2 = fig.add_subplot(111, label="2", frame_on=False)
        ax.plot(ts, signal, color="C0")
        ax.autoscale(tight=True)
        plt.title(channel_name + ": " + key)
        ax.set_xlabel("Time")
        ax.set_ylabel("mV")

        ax2.plot(tf, feature_matrix.T[key], color="red")
        ax2.yaxis.tick_right()
        ax2.autoscale(tight=True)
        ax2.set_xticks([])
        ax2.set_yticks([])
        mng = plt.get_current_fig_manager()
        mng.window.state('zoomed')
        plt.show()

## 2. Basic Features Extracted from EFD and EWT Decomposition

Extracted features include RMS, MAV, Zero Crossing as time domain features, wavelet energy for time-frequency domain feature. All features are extracted based on 5 components of decomposition. The features of each pieces of movement sample will be a  4*5=20 elements vector.

Extract features from EWT

In [75]:
import os
arr = os.listdir(r'C:\Ninapro\pieces_ewt')

In [232]:
full=[]
emp=[]
td=[]
for i in range(12240):
    get_file=os.path.join('C:\\Ninapro\\pieces_ewt\\', arr[i])
    comp=pd.read_csv(get_file,header=None)
    if (len(comp)% 2) == 0:
        pp=int(len(comp)/2)
        a=time_frequency_features_estimation(np.array(comp[0]),pp,pp+1)
        b=time_frequency_features_estimation(np.array(comp[1]),pp,pp+1)
        c=time_frequency_features_estimation(np.array(comp[2]),pp,pp+1)
        d=time_frequency_features_estimation(np.array(comp[3]),pp,pp+1)
        e=time_frequency_features_estimation(np.array(comp[4]),pp,pp+1)
        f=time_features_estimation(np.array(comp[0]),pp,pp+1)
        g=time_features_estimation(np.array(comp[1]),pp,pp+1)
        h=time_features_estimation(np.array(comp[2]),pp,pp+1)
        j=time_features_estimation(np.array(comp[3]),pp,pp+1)
        k=time_features_estimation(np.array(comp[4]),pp,pp+1)
    else:
        pp=int((len(comp)-1)/2)
        a=time_frequency_features_estimation(np.array(comp[0]),pp,pp+1)
        b=time_frequency_features_estimation(np.array(comp[1]),pp,pp+1)
        c=time_frequency_features_estimation(np.array(comp[2]),pp,pp+1)
        d=time_frequency_features_estimation(np.array(comp[3]),pp,pp+1)
        e=time_frequency_features_estimation(np.array(comp[4]),pp,pp+1)
        f=time_features_estimation(np.array(comp[0]),pp,pp+1)
        g=time_features_estimation(np.array(comp[1]),pp,pp+1)
        h=time_features_estimation(np.array(comp[2]),pp,pp+1)
        j=time_features_estimation(np.array(comp[3]),pp,pp+1)
        k=time_features_estimation(np.array(comp[4]),pp,pp+1)
    
    hw=a+b+c+d+e
    #tds=np.concatenate((f,g,h,j,k))
    emp.append(hw)
    td.append(f+g+h+j+k)
    full.append(hw+f+g+h+j+k)
    
    if i%100==0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200


In [233]:
dfewt=pd.DataFrame(full)

Extract features from EFD

In [234]:
arr2 = os.listdir(r'C:\Ninapro\pieces_efd')

In [237]:
full2=[]
emp2=[]
td2=[]
for m in range(12240):
    get_file=os.path.join('C:\\Ninapro\\pieces_ewt\\', arr2[m])
    comp=pd.read_csv(get_file,header=None)
    if (len(comp)% 2) == 0:
        pp=int(len(comp)/2)
        a=time_frequency_features_estimation(np.array(comp[0]),pp,pp+1)
        b=time_frequency_features_estimation(np.array(comp[1]),pp,pp+1)
        c=time_frequency_features_estimation(np.array(comp[2]),pp,pp+1)
        d=time_frequency_features_estimation(np.array(comp[3]),pp,pp+1)
        e=time_frequency_features_estimation(np.array(comp[4]),pp,pp+1)
        f=time_features_estimation(np.array(comp[0]),pp,pp+1)
        g=time_features_estimation(np.array(comp[1]),pp,pp+1)
        h=time_features_estimation(np.array(comp[2]),pp,pp+1)
        j=time_features_estimation(np.array(comp[3]),pp,pp+1)
        k=time_features_estimation(np.array(comp[4]),pp,pp+1)
    else:
        pp=int((len(comp)-1)/2)
        a=time_frequency_features_estimation(np.array(comp[0]),pp,pp+1)
        b=time_frequency_features_estimation(np.array(comp[1]),pp,pp+1)
        c=time_frequency_features_estimation(np.array(comp[2]),pp,pp+1)
        d=time_frequency_features_estimation(np.array(comp[3]),pp,pp+1)
        e=time_frequency_features_estimation(np.array(comp[4]),pp,pp+1)
        f=time_features_estimation(np.array(comp[0]),pp,pp+1)
        g=time_features_estimation(np.array(comp[1]),pp,pp+1)
        h=time_features_estimation(np.array(comp[2]),pp,pp+1)
        j=time_features_estimation(np.array(comp[3]),pp,pp+1)
        k=time_features_estimation(np.array(comp[4]),pp,pp+1)
    
    hw2=a+b+c+d+e
    #tds=np.concatenate((f,g,h,j,k))
    emp2.append(hw2)
    td2.append(f+g+h+j+k)
    full2.append(hw2+f+g+h+j+k)
    
    if m%100==0:
        print(m)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200


In [238]:
dfefd=pd.DataFrame(full2)

In [244]:
dfewt['Name']=pd.DataFrame(arr)
dfefd['Name']=pd.DataFrame(arr2)

In [245]:
dfewt.columns = ['tf1','tf2','tf3','tf4','tf5','rms1','mav1','zc1','rms2','mav2',
                 'zc2','rms3','mav3','zc3','rms4','mav4','zc4','rms5','mav5','zc5','filename']

In [247]:
dfefd.columns = ['tf1','tf2','tf3','tf4','tf5','rms1','mav1','zc1','rms2','mav2',
                 'zc2','rms3','mav3','zc3','rms4','mav4','zc4','rms5','mav5','zc5','filename']

In [248]:
dfewt

,tf1,tf2,tf3,tf4,tf5,rms1,mav1,zc1,rms2,mav2,...,rms3,mav3,zc3,rms4,mav4,zc4,rms5,mav5,zc5,filename
0,2.069935,0.667437,0.084186,0.017825,0.005366,0.000029,0.000019,1533,2.322581e-05,1.510921e-05,...,2.872623e-05,2.008391e-05,157,1.191120e-05,8.023778e-06,100,0.000011,0.000008,66,S10_C10_G10_T1.csv
1,1.840917,0.896476,0.078845,0.019602,0.006194,0.000045,0.000026,1617,3.704691e-05,2.143580e-05,...,3.216675e-05,2.038673e-05,159,1.128716e-05,7.538647e-06,110,0.000018,0.000011,70,S10_C10_G10_T2.csv
2,1.999112,0.117376,0.029379,0.016124,0.007815,0.000037,0.000023,1373,2.186941e-05,1.446362e-05,...,1.234953e-05,9.349297e-06,120,7.515428e-06,5.528849e-06,104,0.000012,0.000008,70,S10_C10_G10_T3.csv
3,1.872467,0.041598,0.037724,0.023905,0.006250,0.000055,0.000031,1173,6.698610e-06,4.504697e-06,...,7.826445e-06,3.578474e-06,123,3.704898e-06,2.589713e-06,111,0.000015,0.000010,63,S10_C10_G10_T4.csv
4,1.942525,0.667698,0.060315,0.013143,0.006804,0.000038,0.000022,1531,2.446644e-05,1.604040e-05,...,2.054532e-05,1.276334e-05,161,6.991268e-06,4.805223e-06,96,0.000012,0.000008,71,S10_C10_G10_T5.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12235,1.512130,1.368342,1.080645,1.223787,1.488795,0.000004,0.000002,2484,3.314914e-06,1.547819e-06,...,2.003781e-06,9.888245e-07,1105,1.786372e-06,9.591628e-07,1006,0.000005,0.000003,502,S9_C9_G9_T2.csv
12236,1.757629,1.254205,1.343601,0.921685,0.207652,0.000002,0.000002,2354,1.087645e-06,7.706140e-07,...,1.032549e-06,8.052422e-07,604,1.087589e-06,8.503389e-07,397,0.000002,0.000001,214,S9_C9_G9_T3.csv
12237,1.807096,1.260688,0.986512,0.577975,0.146439,0.000004,0.000002,2053,1.482122e-06,9.269536e-07,...,7.844239e-07,6.060511e-07,431,1.173935e-06,7.992445e-07,331,0.000001,0.000001,185,S9_C9_G9_T4.csv
12238,2.036000,1.191392,0.994418,0.722345,0.237137,0.000003,0.000002,1888,6.385321e-07,4.415922e-07,...,6.477366e-07,5.217466e-07,431,6.826907e-07,5.201186e-07,369,0.000002,0.000001,221,S9_C9_G9_T5.csv


## 3. Save result to csv files and have a basic cleaning in Excel.

In [249]:
dfewt.to_csv(r'C:\Users\16468\Desktop\STROKE literatures\ewtfeatures.csv')
dfefd.to_csv(r'C:\Users\16468\Desktop\STROKE literatures\efdfeatures.csv')

In [388]:
dfewt=pd.read_csv(r'C:\Users\16468\Desktop\STROKE literatures\ewtfeatures.csv')
dfefd=pd.read_csv(r'C:\Users\16468\Desktop\STROKE literatures\efdfeatures.csv')

In [389]:
dfewt3=dfewt[dfewt.Channel==3]
dfewt3

,Unnamed: 0,tf1,tf2,tf3,tf4,tf5,rms1,mav1,zc1,rms2,...,rms4,mav4,zc4,rms5,mav5,zc5,Subject,Channel,Gesture,Rep
510,510,1.467504,1.245125,1.335687,1.372402,1.599611,0.000053,0.000018,2713,2.400000e-05,...,2.950000e-05,1.200000e-05,1224,0.000044,0.000018,621,S10,3,10,1
511,511,1.578819,1.345419,1.236923,1.153932,1.389474,0.000011,0.000004,2341,4.720000e-06,...,3.320000e-06,1.680000e-06,909,0.000008,0.000004,484,S10,3,10,2
512,512,1.437396,1.377416,1.247557,1.147648,1.369295,0.000017,0.000005,2623,1.410000e-05,...,6.370000e-06,2.540000e-06,894,0.000015,0.000006,477,S10,3,10,3
513,513,1.472270,1.297513,1.368006,1.362204,1.583179,0.000012,0.000004,2688,6.670000e-06,...,6.560000e-06,2.920000e-06,1207,0.000014,0.000006,578,S10,3,10,4
514,514,1.559955,1.463228,1.360397,1.300197,1.302016,0.000009,0.000003,2452,1.730000e-06,...,4.970000e-06,2.400000e-06,1098,0.000010,0.000005,475,S10,3,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11623,11623,1.527558,1.354057,1.369037,1.314635,0.800573,0.000002,0.000001,2644,7.760000e-07,...,1.600000e-06,1.030000e-06,876,0.000002,0.000002,355,S9,3,9,2
11624,11624,1.980725,1.177507,1.095881,1.021868,0.515432,0.000004,0.000002,1902,6.020000e-07,...,4.730000e-07,3.540000e-07,439,0.000002,0.000001,280,S9,3,9,3
11625,11625,1.757122,1.159972,1.198177,1.380585,0.827050,0.000003,0.000002,2239,7.020000e-07,...,1.350000e-06,8.460000e-07,632,0.000002,0.000001,327,S9,3,9,4
11626,11626,1.533022,1.368880,1.311341,1.304841,1.056757,0.000003,0.000001,2673,1.110000e-06,...,1.670000e-06,8.950000e-07,890,0.000002,0.000002,381,S9,3,9,5


### 3.1 Take Channel Three and 3 Guestures for Classification 
#### EWT attempt

In [602]:
dfewt3=dfewt[dfewt.Channel==3]
dfewt3=dfewt3[(dfewt3.Gesture == 1)|(dfewt3.Gesture == 5)|(dfewt3.Gesture == 8)|(dfewt3.Gesture == 10)]

In [603]:
dfewt3

,Unnamed: 0,tf1,tf2,tf3,tf4,tf5,rms1,mav1,zc1,rms2,...,rms4,mav4,zc4,rms5,mav5,zc5,Subject,Channel,Gesture,Rep
510,510,1.467504,1.245125,1.335687,1.372402,1.599611,0.000053,0.000018,2713,0.000024,...,0.000029,1.200000e-05,1224,0.000044,0.000018,621,S10,3,10,1
511,511,1.578819,1.345419,1.236923,1.153932,1.389474,0.000011,0.000004,2341,0.000005,...,0.000003,1.680000e-06,909,0.000008,0.000004,484,S10,3,10,2
512,512,1.437396,1.377416,1.247557,1.147648,1.369295,0.000017,0.000005,2623,0.000014,...,0.000006,2.540000e-06,894,0.000015,0.000006,477,S10,3,10,3
513,513,1.472270,1.297513,1.368006,1.362204,1.583179,0.000012,0.000004,2688,0.000007,...,0.000007,2.920000e-06,1207,0.000014,0.000006,578,S10,3,10,4
514,514,1.559955,1.463228,1.360397,1.300197,1.302016,0.000009,0.000003,2452,0.000002,...,0.000005,2.400000e-06,1098,0.000010,0.000005,475,S10,3,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11617,11617,1.564848,1.466830,1.294707,1.263057,1.364767,0.000006,0.000003,2429,0.000003,...,0.000003,1.740000e-06,958,0.000005,0.000003,484,S9,3,8,2
11618,11618,1.543082,1.378127,1.378221,1.269905,1.367052,0.000006,0.000003,2584,0.000003,...,0.000004,2.320000e-06,1005,0.000005,0.000003,500,S9,3,8,3
11619,11619,1.693320,1.297514,1.330496,1.172576,1.388889,0.000008,0.000004,2249,0.000003,...,0.000003,1.380000e-06,884,0.000005,0.000003,486,S9,3,8,4
11620,11620,1.689198,1.346314,1.256305,1.151564,1.392977,0.000006,0.000003,2304,0.000002,...,0.000001,8.070000e-07,916,0.000005,0.000003,465,S9,3,8,5


In [604]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

x = dfewt3.drop(['Unnamed: 0','Subject','Rep','Gesture','Channel','tf2','tf3','tf4','tf5'],axis=1)
y= dfewt3['Gesture']
#x = tr1.drop(['Index','Subject','Gesture','Rep','cls'],axis=1)
#y = tr1['cls']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=12)

In [605]:
rfc = RandomForestClassifier()

In [606]:
rfc.fit(x_train, y_train)

RandomForestClassifier()

In [607]:
rfc.predict(x_train)

array([ 1,  1, 10,  5,  8,  8,  5,  5, 10,  1,  1,  5, 10, 10,  8,  1,  1,
        1,  5,  8,  1,  5, 10,  1,  5,  8,  5,  5,  1, 10,  1,  1,  1,  5,
        1,  8,  8,  1,  5,  5,  8,  5,  1,  5,  8,  5,  8,  5,  1,  8,  8,
        8,  5, 10,  1, 10,  8, 10, 10,  8,  8,  5,  5, 10,  5,  8,  1,  5,
        8,  5, 10, 10,  1,  8, 10, 10,  1,  1,  1, 10,  5, 10,  1,  5,  8,
        8,  8,  1,  8, 10,  8,  8, 10,  8,  1,  5,  8, 10, 10,  5,  1,  1,
        5,  8,  1,  8, 10,  5,  1,  1, 10,  5,  8,  8,  8,  8,  5, 10,  1,
       10,  1, 10,  8, 10,  5,  5,  5, 10, 10,  5, 10,  8,  1, 10, 10,  8,
        8,  5,  5,  8, 10,  8,  1,  5,  1,  5,  5,  1,  8,  1, 10,  8,  1,
        8, 10,  8,  5,  5, 10,  8,  1, 10, 10, 10,  1,  5,  1, 10],
      dtype=int64)

In [608]:
rfc.score(x_test, y_test)

0.5138888888888888

In [609]:
rfc.score(x_train, y_train)

1.0

#### EFD attempt

In [617]:
dfefd3=dfefd[dfefd.Channel==3]
dfefd3=dfefd3[(dfefd3.Gesture == 1)|(dfefd3.Gesture == 5)|(dfefd3.Gesture == 8)|(dfefd3.Gesture == 10)]

In [618]:
dfefd3

,Unnamed: 0,tf1,tf2,tf3,tf4,tf5,rms1,mav1,zc1,rms2,...,rms4,mav4,zc4,rms5,mav5,zc5,Subject,Channel,Gesture,Rep
510,510,1.467504,1.245125,1.335687,1.372402,1.599611,0.000053,0.000018,2713,0.000024,...,0.000029,1.200000e-05,1224,0.000044,0.000018,621,S10,3,10,1
511,511,1.578819,1.345419,1.236923,1.153932,1.389474,0.000011,0.000004,2341,0.000005,...,0.000003,1.680000e-06,909,0.000008,0.000004,484,S10,3,10,2
512,512,1.437396,1.377416,1.247557,1.147648,1.369295,0.000017,0.000005,2623,0.000014,...,0.000006,2.540000e-06,894,0.000015,0.000006,477,S10,3,10,3
513,513,1.472270,1.297513,1.368006,1.362204,1.583179,0.000012,0.000004,2688,0.000007,...,0.000007,2.920000e-06,1207,0.000014,0.000006,578,S10,3,10,4
514,514,1.559955,1.463228,1.360397,1.300197,1.302016,0.000009,0.000003,2452,0.000002,...,0.000005,2.400000e-06,1098,0.000010,0.000005,475,S10,3,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11617,11617,1.564848,1.466830,1.294707,1.263057,1.364767,0.000006,0.000003,2429,0.000003,...,0.000003,1.740000e-06,958,0.000005,0.000003,484,S9,3,8,2
11618,11618,1.543082,1.378127,1.378221,1.269905,1.367052,0.000006,0.000003,2584,0.000003,...,0.000004,2.320000e-06,1005,0.000005,0.000003,500,S9,3,8,3
11619,11619,1.693320,1.297514,1.330496,1.172576,1.388889,0.000008,0.000004,2249,0.000003,...,0.000003,1.380000e-06,884,0.000005,0.000003,486,S9,3,8,4
11620,11620,1.689198,1.346314,1.256305,1.151564,1.392977,0.000006,0.000003,2304,0.000002,...,0.000001,8.070000e-07,916,0.000005,0.000003,465,S9,3,8,5


In [619]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

x = dfefd3.drop(['Unnamed: 0','Subject','Rep','Gesture','Channel','tf1','tf2','tf3','tf4'],axis=1)
y= dfefd3['Gesture']
#x = tr1.drop(['Index','Subject','Gesture','Rep','cls'],axis=1)
#y = tr1['cls']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=12)

In [620]:
rfc2 = RandomForestClassifier()
rfc2.fit(x_train, y_train)

RandomForestClassifier()

In [621]:
rfc2.predict(x_train)

array([ 1,  1, 10,  5,  8,  8,  5,  5, 10,  1,  1,  5, 10, 10,  8,  1,  1,
        1,  5,  8,  1,  5, 10,  1,  5,  8,  5,  5,  1, 10,  1,  1,  1,  5,
        1,  8,  8,  1,  5,  5,  8,  5,  1,  5,  8,  5,  8,  5,  1,  8,  8,
        8,  5, 10,  1, 10,  8, 10, 10,  8,  8,  5,  5, 10,  5,  8,  1,  5,
        8,  5, 10, 10,  1,  8, 10, 10,  1,  1,  1, 10,  5, 10,  1,  5,  8,
        8,  8,  1,  8, 10,  8,  8, 10,  8,  1,  5,  8, 10, 10,  5,  1,  1,
        5,  8,  1,  8, 10,  5,  1,  1, 10,  5,  8,  8,  8,  8,  5, 10,  1,
       10,  1, 10,  8, 10,  5,  5,  5, 10, 10,  5, 10,  8,  1, 10, 10,  8,
        8,  5,  5,  8, 10,  8,  1,  5,  1,  5,  5,  1,  8,  1, 10,  8,  1,
        8, 10,  8,  5,  5, 10,  8,  1, 10, 10, 10,  1,  5,  1, 10],
      dtype=int64)

In [622]:
rfc2.score(x_test, y_test)

0.5555555555555556

In [623]:
rfc2.score(x_train, y_train)

1.0